## Base Imports and Tensorflow

In [1]:
# import base libraries and Tensoflow
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys


%matplotlib inline

## Setup azure workspace for Data & Labels

In [2]:
#import for Azure data
from azureml.core import Dataset, Datastore, Workspace, Experiment, ComputeTarget
# import azureml.contrib.dataset
# from azureml.contrib.dataset import FileHandlingOption, LabeledDatasetTask
from azureml.core.authentication import InteractiveLoginAuthentication
import os
from pathlib import Path, PurePosixPath, PureWindowsPath

In [3]:
df_lab = pd.read_csv('../datasets/animals/labels.csv')
df_lab.head()

,FilePath,Label
0,cats/cats_00072.jpg,cat
1,cats/cats_00073.jpg,cat
2,cats/cats_00074.jpg,cat
3,cats/cats_00075.jpg,cat
4,cats/cats_00076.jpg,cat


In [4]:
# Set data directory to where images are
data_dir = '../datasets/animals'

In [5]:
# get file paths and labels from label df
file_paths_and_labels_df = df_lab

In [6]:
file_paths_and_labels_df.head()

,FilePath,Label
0,cats/cats_00072.jpg,cat
1,cats/cats_00073.jpg,cat
2,cats/cats_00074.jpg,cat
3,cats/cats_00075.jpg,cat
4,cats/cats_00076.jpg,cat


In [7]:
file_paths_and_labels_df.shape

(2850, 2)

In [8]:
# TODO SET RANDOM SEEDS

In [9]:
# Model Params

# Image height & width
img_height = 128
img_width = 128
img_depth = 3

IMG_SHAPE = (img_height, img_width, img_depth)


# initialize learning rate, batch size, and number of epochs
INIT_LR =  0.001
BATCH_SIZE = 32
NUM_EPOCHS = 300

# initialize the label names if needed
num_classes = 3


## Load Images

In [10]:
sys.path.insert(0, './helpers')

In [11]:
from image_loader import train_generator


In [12]:
gen = train_generator(img_height,
                      img_width,BATCH_SIZE,
                      data_dir,
                      file_paths_and_labels_df)

Found 2280 validated image filenames belonging to 3 classes.
Found 570 validated image filenames belonging to 3 classes.


In [13]:
# gen.image_check()

## Create Model

In [14]:
from model import resnet

2.0.0


In [15]:
res = resnet(num_classes,INIT_LR,NUM_EPOCHS,IMG_SHAPE)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [16]:
train_data = gen.train_generator
val_data = gen.validation_generator

In [19]:
H = res.train(BATCH_SIZE,
              NUM_EPOCHS,
              train_data,
              val_data)

Epoch 1/300
71/71 [==============================] - 18s 252ms/step - loss: 0.1420 - accuracy: 0.9480 - val_loss: 0.2376 - val_accuracy: 0.9191
Epoch 2/300
20/71 [=======>......................] - ETA: 11s - loss: 0.1186 - accuracy: 0.9481

KeyboardInterrupt: 

In [ ]:
break

In [ ]:
# model.save('outputs/weights.hdf5')

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [ ]:
break

In [ ]:
#plt.savefig('foo.png')